# **Initial setup**

Install Bambu and required packages:

In [ ]:
!echo "deb http://ppa.launchpad.net/git-core/ppa/ubuntu $(cat /etc/os-release | grep UBUNTU_CODENAME | sed 's/.*=//g') main" >> /etc/apt/sources.list.d/git-core.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A1715D88E1DF1F24
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git iverilog verilator wget
!wget https://release.bambuhls.eu/appimage/bambu-fpl22.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!ln -sf $PWD/bambu-*.AppImage /bin/tree-panda-gcc
!ln -sf $PWD/bambu-*.AppImage /bin/clang-12
!git clone --depth 1 --filter=blob:none --sparse https://github.com/ferrandi/PandA-bambu.git --branch doc/fpl22
%cd PandA-bambu
!git sparse-checkout set documentation/tutorial_fpl_2022 
%cd ..
!mv PandA-bambu/documentation/tutorial_fpl_2022/ bambu-tutorial

# **Productive HLS with Bambu**


## **Exercise 1**

Have a look at the C code in /content/bambu-tutorial/01-introduction/Exercise1/icrc.c

Launch bambu:

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise1
!bambu icrc.c --top-fname=icrc1

Inspect the generated Verilog file in the explorer tab on the left (icrc1.v)


Take a brief look at the available Bambu options:


In [ ]:
!bambu --help

Modify the command line to change the amount of debug information displayed, and generate VHDL instead of Verilog code:


In [ ]:
!bambu icrc.c --top-fname=icrc1 -wH

## **Exercise 2**

We remain on the same input C code as before, let's add co-simulation:


In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise1
!bambu icrc.c --top-fname=icrc1 --simulate --simulator=VERILATOR

We did not specify any input values. Inspect what Bambu generated automatically:

In [ ]:
!cat test.xml

You can find the actual testbench in HLS_output/simulation.

## **Exercise 3**

Implement and synthesize a module that returns the minimum and maximum value in an array of integers with arbitrary size.

Write the input C code starting from this snippet:

In [ ]:
%%writefile /content/bambu-tutorial/01-introduction/Exercise2/minmax.c
void min_max(int input[10], int* out_max)
{
   int local_max = input[0];
   int i = 0;
   for(i = 0; i < 10; i++)
   {
      if(input[i] > local_max)
      {
         local_max = input[i];
      }
   }
   *out_max = local_max;
}

Write a testbench to test arrays with different elements and different sizes.

Start from the XML snippet below **(parameter names need to correspond to function arguments in your code)**:

In [ ]:
%%writefile /content/bambu-tutorial/01-introduction/Exercise2/testbench.xml
<?xml version="1.0"?>
<function>
   <testbench input="{0,1,2,3,4}" num_elements="5" out_max="{0}" out_min="{0}"/>
   <testbench input="{15,10,5}" num_elements="3" out_max="{15}" out_min="{5}"/>
</function>

Synthesize with Bambu and simulate with Verilator **(double check the command line if you changed file/function names)**:

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise2/
!bambu minmax.c --top-fname=min_max --generate-tb=testbench.xml --simulate --simulator=VERILATOR

What happens if you pass an array with a different number of elements than what is specified in num_elements? **(remember to fix the XML file afterwards, we will need it again)**

## **Exercise 4**

Bambu can synthesize accelerators described in an LLVM IR through the Clang frontend.

Synthesize /content/bambu-tutorial/01-introduction/Exercise3/matmul.ll, which contains a matrix multiplication kernel generated by [soda-opt](https://gitlab.pnnl.gov/sodalite/soda-opt):

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise3/
!bambu matmul.ll --top-fname=main_kernel --generate-tb=test.xml --simulate --simulator=VERILATOR --compiler=I386_CLANG12

Note: kernels generated by soda-opt require at least Clang 10.

## **Exercise 5**

Let's go back to the C code that finds minumim and maximum in an array of numbers, and compare performance across different target platforms and frequencies.

Start from the given command and modify the options appropriately to test the following combinations:


*   xc4vlx100-10ff1513 (Xilinx Virtex 4) – 66MHz
*   5SGXEA7N2F45C1 (Intel Stratix V) – 200MHz
*   xc7vx690t-3ffg1930-VVD (Xilinx Virtex 7) – 100MHz
*   xc7vx690t-3ffg1930-VVD (Xilinx Virtex 7) – 333MHz
*   xc7vx690t-3ffg1930-VVD (Xilinx Virtex 7) – 400MHz
*   nangate45 (45nm ASIC) – 200MHz



In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise2
!bambu minmax.c --top-fname=min_max --device-name=xc4vlx100-10ff1513 --clock-period=15 --no-iob --simulate --simulator=VERILATOR --generate-tb=testbench.xml

Look also at the different simulation and synthesis scripts generated by Bambu.

## **Exercise 6**

Ask Bambu to print a C verion of its internal IR and all relevant graphs:

In [ ]:
!bambu minmax.c --top-fname=min_max --pretty-print=out.c --print-dot

Look at /content/bambu-tutorial/01-introduction/Exercise2/out.c and then print the FSM graph:

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/min_max/fsm.dot')

## **Exercise 7**

Bambu automatically enables the synthesis of function proxies to save area.

Synthesize the dummy example in /content/bambu-tutorial/01-introduction/Exercise4/proxies.c, and then look for the PROXY_PREF_funcC module in the generated Verilog:


In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise4/
!bambu proxies.c --top-fname=funcA

Note: floating point operations are synthesized as functions! Look at the kernels in /content/bambu-tutorial/01-introduction/Exercise5a/helm.c (adapted from the computational fluid dynamics applications in the [EVEREST](https://everest-h2020.eu/) project). Synthesize one of them:

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise5a/
!bambu helm.c --top-fname=helm_naive --simulate --simulator=VERILATOR --generate-tb=test.xml --compiler=I386_CLANG6

Run the same synthesis disabling function proxies:

In [ ]:
!bambu helm.c --top-fname=helm_naive --simulate --simulator=VERILATOR --generate-tb=test.xml --compiler=I386_CLANG6 --disable-function-proxy

## **Exercise 8** 
Generate an accelerator with outer loop vectorization, try different vector sizes (-fopenmp-simd) and see how performance changes.

In [ ]:
%cd /content/bambu-tutorial/04-simd/Exercise1/
!bambu --compiler=I386_GCC49 --device-name=5SGXEA7N2F45C1 --simulate -fwhole-program -fno-delete-null-pointer-checks --clock-period=10 --experimental-setup=BAMBU-BALANCED-MP -fdisable-tree-cunroll -fdisable-tree-ivopts --param max-inline-insns-auto=1000 histogram.c -fopenmp-simd=1

## **Other examples**

You can play around with a set of other examples that contain different applications and showcase different features of Bambu.

 - /content/bambu-tutorial/01-introduction/Exercise5: LU decomposition
 - /content/bambu-tutorial/01-introduction/Exercise6: integration of IPs written in Verilog
 - /content/bambu-tutorial/01-introduction/Exercise7: sorting algorithm
 - /content/bambu-tutorial/01-introduction/Exercise8: cryptographic core
 - /content/bambu-tutorial/01-introduction/Exercise9: search and insertion in a binary tree


# **Optimizations**

## **Exercise 1** 

Modify Bambu options to evaluate the effect of:


*   different levels of optimization (-O0, -O1, -O2, -O3, -Os)
*   vectorization (-ftree-vectorize)
*   inlining (-finline-limit=100000)
*   different frontend compilers (--compiler={I386_GCC49|I386_GCC7|I386_CLANG6|I386_CLANG12})

#### **ADPCM from CHStone benchmark suite**
Adaptive Diferential Pulse-Code Modulation is an algorithm used to perform audio compression (mainly in telephony). It is part of the CHStone benchmark suite for C-based HLS tools.
* Yuko Hara, Hiroyuki Tomiyama, Shinya Honda and Hiroaki Takada, "Proposal and Quantitative Analysis of the CHStone Benchmark Program Suite for Practical C-based High-level Synthesis", *Journal of Information Processing*, Vol. 17, pp.242-254, (2009).

In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise1/
!bambu adpcm.c -O0 --simulate

## **Exercise 2** 

Use the command that yielded the best result in Exercise 1 and verify if SDC scheduling can introduce further improvements.

* -s or --speculative-sdc-scheduling

In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise1/
!bambu adpcm.c -O0 --simulate

## **Exercise 3**

Modify Bambu options to evaluate the effect of different integer division implementations.

--hls-div=<method\>
* none  - use a HDL based pipelined restoring division
* nr1   - use a C-based non-restoring division with unrolling factor equal to 1 (default)
* nr2   - use a C-based non-restoring division with unrolling factor equal to 2
* NR    - use a C-based Newton-Raphson division
* as    - use a C-based align divisor shift dividend method

#### **FPDiv from CHStone**
Soft floating-point division implementation from the CHStone benchmark suite for C-based HLS.
* Yuko Hara, Hiroyuki Tomiyama, Shinya Honda and Hiroaki Takada, "Proposal and Quantitative Analysis of the CHStone Benchmark Program Suite for Practical C-based High-level Synthesis", *Journal of Information Processing*, Vol. 17, pp.242-254, (2009).


In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise3/
!bambu dfdiv.c --simulate --clock-period=15 --hls-div=none

## **Exercise 4**
Not all target devices may expose support for the synthesis of the whole set of Bambu IR instructions. If this is the case, Bambu takes care of replacing unsupported IR instructions with function calls which implement the same functionality using supported functional units only.

As an example, NanoXplore nx1h140tsp FPGA board does not support 64x64 integer multiplication, thus Bambu will replace this simple IR instruction with a function call to a different multiplier implementation which exploits smaller multipliers to build the final result.

In [ ]:
%%writefile /content/bambu-tutorial/03-optimizations/Exercise4/module.c
long long func_replace(long long a, long long b)
{
   return a * b;
}

Make sure you run the above cell before launching Bambu in the following.


In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise4/
!bambu module.c --top-fname=func_replace --device-name=nx1h140tsp --generate-tb=a=4,b=5 --simulate --panda-parameter=function-opt=0 --print-dot

Once the synthesis has completed it is possible to have a look at the initial and final call graph. In the former no function calls are present, while in the latter a __umul64 function call has been added. This function implements the 64x64 integer multiplication by means of smaller 32x32 multipliers.

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/call_graph.dot')

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/call_graph_final.dot')

Also it is possible to have a look at the __umul64 FSM graph to see how the 64x64 multiplication logic is implemented.

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/__umul64/fsm.dot')

## **Exercise 5** 
Bambu expose a complete support for floating-point arithemtic and all libm functions.
In the following you can define any arbitrary floating-point computation and take a look at the generated design structure.

As an example, try to write a C implementation that compute the following:

# $awesome\_math(a,b,c) = acos(\frac{a^2+b^2-c^2}{2ab})$

Experiment with single and double precision data types, different softfloat and libm implementations offered by bambu.

Start by editing this code and then try different bambu options:
* Different floating-point arithmetic implementations (--softfloat, --soft-fp, --flopoco)
* Different libm implementations (--libm-std-rounding)
* Different square implementation (pow, simple multiplication)

In [ ]:
%%writefile /content/bambu-tutorial/03-optimizations/Exercise5/module.c
#include <math.h>
float awesome_math(float a, float b, float c)
{
   return a * b + c;
}

Make sure you run the above cell after you write the C implementation inside, so that the file is updated, then launch Bambu to perform the synthesis.

In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise5/
!bambu module.c -O3 -lm --simulate --top-fname=awesome_math --generate-tb="a=3.0,b=4.0,c=5.0" --panda-parameter=function-opt=0 --print-dot

After the synthesis has completed it is possible to observe how the floating-point operations have been converted to function calls to the internal Bambu arithmetic cores and libm implementation.

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/call_graph_final.dot')

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/__float_adde8m23b_127nih/fsm.dot')

## **Excercise 6**
Further support for custom floating-point arithmetic is also available in Bambu.

The user may define custom floating-point types through the command line API and the HLS engine will take care of the conversions and implementation starting from standard float/double types defined in the C implementation.

```
--fp-subnormal
        Enable the soft-based implementation of floating-point operations with
        subnormals support.

--fp-exception-mode=<ieee|saturation|overflow>
    Set the soft-based exception handling mode:
          ieee    - IEEE754 standard exceptions (default)
       saturation - Inf is replaced with max value, Nan becomes undefined behaviour
        overflow  - Inf and Nan results in undefined behaviour

--fp-rounding-mode=<nearest_even|truncate>
    Set the soft-based rounding handling mode:
       nearest_even - IEEE754 standard rounding mode (default)
          truncate  - No rounding is applied

--fp-format=<func_name>*e<exp_bits>m<frac_bits>b<exp_bias><rnd_mode><exc_mode><?spec><?sign>
    Define arbitrary precision floating-point format by function (use comma separated
    list for multiple definitions). (i.e.: e8m27b-127nihs represent IEEE754 single precision FP)
       func_name - Set arbitrary floating-point format for a specific function (using
                   @ symbol here will resolve to the top function)
                   (Arbitrary floating-point format will apply to specified function
                   only, use --propagate-fp-format to extend it to called functions)
        exp_bits - Number of bits used by the exponent
       frac_bits - Number of bits used by the fractional value
        exp_bias - Bias applied to the unsigned value represented by the exponent bits
        rnd_mode - Rounding mode (exclusive option):
                      n - nearest_even: IEEE754 standard rounding mode
                      t - truncate    : no rounding is applied
        exc_mode - Exception mode (exclusive option):
                      i - ieee      : IEEE754 standard exceptions
                      a - saturation: Inf is replaced with max value, Nan becomes undefined behaviour
                      o - overflow  : Inf and Nan results in undefined behaviour
          spec   - Floating-point specialization string (multiple choice):
                      h - hidden one: IEEE754 standard representation with hidden one
                      s - subnormals: IEEE754 subnormal numbers
          sign   - Static sign representation (exclusive option):
                        - IEEE754 dynamic sign is used if omitted
                      1 - all values are considered as negative numbers
                      0 - all values are considered as positive numbers

--fp-format=inline-math
    The "inline-math" flag may be added to fp-format option to force floating-point
    arithmetic operators always inline policy

--fp-format=inline-conversion
    The "inline-conversion" flag may be added to fp-format option to force floating-point
    conversion operators always inline policy

--fp-format-interface
    User-defined floating-point format is applied to top interface signature if required
    (default modifies top function body only)

--fp-format-propagate
    Propagate user-defined floating-point format to called function when possible
```

In [ ]:
%%writefile /content/bambu-tutorial/03-optimizations/Exercise6/module.c
float user_fp(float a, float b, float c)
{
   return a * b + c;
}

As an example, the above code may be synthesized as is with standard single-precision floating-point arithmetic or it may be synthesized replacing float with half-precision encoding, thus a 16-bit floating-point format through the following.

In [ ]:
%cd /content/bambu-tutorial/03-optimizations/Exercise6/
!bambu module.c -O3 -lm --simulate --top-fname=user_fp --fp-format=user_fp*e5m10b-16nih --fp-format-interface --generate-tb="a=3.0,b=4.0,c=5.0" --print-dot

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/call_graph_final.dot')

# AXI

## **Exercise 1**
Start by writing a C function called read that simply reads a number from an AXI bus and returns the value that is retrieved from the bus.


In [ ]:
%%writefile /content/bambu-tutorial/04-axi/Exercise1/module.c


Now add the interface infer flag to the bambu command and execute.

In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise1/
!bambu module.c --top-fname=read --compiler=I386_CLANG6

Open the generated Verilog file and look for the top module, called read. Notice the presence of the AXI signals and how their size matches the size of the data.



In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise1/
cat module.v

Finally, launch the simulation and check that everything works properly.

In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise1/
!bambu module.c --top-fname=read --compiler=I386_CLANG6 --generate-interface=INFER --generate-tb="data={96}" --simulator=VERILATOR --simulate -v4

## **Exercise 2**

Consider the following code, that adds up all of the n elements of a vector v. Edit the code so that both the number of elements and the elements of the vector are read from an external memory through an AXI bus.

In [ ]:
%%writefile /content/bambu-tutorial/04-axi/Exercise2/module.c

int sum(int* v, unsigned* n)
{
   int sum = 0;

   for(unsigned i = 0; i < *(n); i++)
   {
      sum += v[i];
   }

   return sum;
}

Launch bambu and simulate the execution.

In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise2/
!bambu module.c --top-fname=sum --compiler=I386_CLANG6 --generate-interface=INFER --generate-tb="data={1, 5, -6, 2, 8},n={5}" --simulator=VERILATOR --simulate -v4

## **Exercise 3**

Let's consider the following code, that computes the maximum among the elements of a vector. We want to read the number of elements of the vector and its data from an AXI bus, however, instead of returning the result, we then want to write the result to an external memory available over a different AXI bus. In order for bambu to generate the module according to our needs, we will need to provide additional information through "bundle", an optional parameter of the pragma directive.
With the addition of the optional parameter, the directive becomes:

#pragma HLS_interface <variable_name> m_axi direct bundle=<bundle_name>

By associating different variables to the same bundle name, we are telling bambu that they will use the same bus. When different names are used, bambu will generate a bus for each bundle.



In [ ]:
%%writefile /content/bambu-tutorial/04-axi/Exercise3/module.c

void maxNumbers(int* a, unsigned int* n_ptr, int* res)

{
   unsigned i;
   int result;
   unsigned int n = *n_ptr;

   if(n == 0)
   {
      *res = (int)(1 << 31);
      return;
   }
   result = a[0];
   for(i = 1; i < n; ++i)
      result = result < a[i] ? a[i] : result;
   *res = result;
}

Once again, we can run bambu with the same command and perform a simulation.

In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise3/
!bambu module.c --top-fname=maxNumbers --compiler=I386_CLANG6 --generate-interface=INFER --generate-tb="a={21, 8, -3, -90},n_ptr={4}" --simulator=VERILATOR --simulate -v4

If we open the module definition, we can actually check that two AXI buses are defined and used.

In [ ]:
%cd /content/bambu-tutorial/04-axi/Exercise3/
!cat module.v